In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install numpy==1.16.1

In [ ]:
import tensorflow as tf
from tensorflow.contrib import learn
import pandas as pd
import numpy as np
import datetime
import pickle as pkl
import time
import json
import os
import sys
import csv
os.chdir('/content/drive/My Drive')

Load data

In [ ]:
gold_contents = np.load('/content/drive/My Drive/Prompt-related Model/word embedding/set1_gold_data.npy')
gold_labels = np.load('/content/drive/My Drive/Prompt-related Model/word embedding/set1_gold_label.npy')
# Fake data
fake_contents = np.load('/content/drive/My Drive/Prompt-related Model/word embedding/set1_fake_data.npy')
fake_labels = np.load('/content/drive/My Drive/Prompt-related Model/word embedding/set1_fake_label.npy')

In [ ]:
# Normalize score
min_val = 0
max_val = max(gold_labels)
gold_labels = (gold_labels - min_val) / (max_val - min_val)

In [ ]:
# Generate sequence length --> bi-lstm sequence length
lengths = []
for i in range(len(gold_contents)):
    lengths.append(len(gold_contents[i]))
for i in range(len(fake_contents)):
    lengths.append(len(fake_contents[i]))
lengths = np.array(lengths)
max_length = max(lengths)

In [ ]:
gold_lengths = lengths[0:1783]
fake_lengths = lengths[1783:]

In [ ]:
# Extend train_content, test_content according to the max length of essay
tmp = [0] * 768
for i in range(len(gold_contents)):
    length = len(gold_contents[i])
    if length < max_length:
        tmp_ = np.tile(tmp, (max_length - length, 1))
        gold_contents[i] = np.append(gold_contents[i], tmp_, axis=0)
for i in range(len(fake_contents)):
    length = len(fake_contents[i])
    if length < max_length:
        tmp_ = np.tile(tmp, (max_length - length, 1))
        fake_contents[i] = np.append(fake_contents[i], tmp_, axis=0)

In [ ]:
gold_data = [gold_contents[0]]
for i in range(1, len(gold_contents)):
    gold_data = np.concatenate((gold_data, [gold_contents[i]]), axis=0)
gold_contents = gold_data

fake_data = [fake_contents[0]]
for i in range(1, len(fake_contents)):
    fake_data = np.concatenate((fake_data, [fake_contents[i]]), axis=0)
fake_contents = fake_data

In [ ]:
min_size = 128
for i in range(2,13):
    print ("****", i)
    curr_size = np.sum(gold_labels*(max_val-min_val)+min_val==i)
    idxs = np.where((gold_labels*(max_val-min_val)+min_val==i)==True)

    if curr_size >= min_size:
        print ("pass")
        continue  

    else:
        exp = min_size // curr_size
        if (exp - 1 > 0):
            tmp_data = np.tile(gold_contents[idxs], (exp-1,1,1))
            tmp_labels = np.tile(gold_labels[idxs], exp-1)
            tmp_lengths = np.tile(gold_lengths[idxs], exp-1)

            gold_contents = np.concatenate((gold_contents, tmp_data), axis = 0)
            gold_labels = np.concatenate((gold_labels, tmp_labels), axis = 0)
            gold_lengths = np.concatenate((gold_lengths, tmp_lengths), axis = 0)

In [ ]:
# Shuffle
permutation = np.random.permutation(fake_labels.shape[0])[0: min_size]
shuffled_fake_content = fake_contents[permutation, :, :]
shuffled_fake_labels = fake_labels[permutation]
shuffled_fake_length = fake_lengths[permutation]

5-Fold Cross Validation

In [ ]:
# Shuffle
shuffled_content = np.concatenate((gold_contents, shuffled_fake_content), axis = 0)
shuffled_labels = np.concatenate((gold_labels, shuffled_fake_labels), axis = 0)
shuffled_length = np.concatenate((gold_lengths, shuffled_fake_length), axis = 0)

permutation = np.random.permutation(shuffled_labels.shape[0])
shuffled_content = shuffled_content[permutation, :, :]
shuffled_labels = shuffled_labels[permutation]
shuffled_length = shuffled_length[permutation]

In [ ]:
for i in range(4):
    dataset = np.concatenate((shuffled_content[0 : 5*unit], shuffled_content[0 : 4*unit]), axis=0) 
    label = np.concatenate((shuffled_labels[0 : 5*unit], shuffled_labels[0 : 4*unit]), axis=0) 
    length = np.concatenate((shuffled_length[0 : 5*unit], shuffled_length[0 : 4*unit]), axis=0) 

In [ ]:
np.save('/content/drive/My Drive/Prompt-related Model/data/dataset', dataset)
np.save('/content/drive/My Drive/Prompt-related Model/data/label', label)
np.save('/content/drive/My Drive/Prompt-related Model/data/length', length)

In [ ]:
dataset = np.load('/content/drive/My Drive/Prompt-related Model/data/dataset.npy')
label = np.load('/content/drive/My Drive/Prompt-related Model/data/label.npy')
length = np.load('/content/drive/My Drive/Prompt-related Model/data/length.npy')

In [ ]:
# Split data
size = len(label)
unit = int(size / 9)

In [ ]:
# 5 sets：each set --> 60% train dataset，20% development dataset，20% test dataset
train_dataset = []
train_label = []
train_length = []

dev_dataset = []
dev_label = []
dev_length = []

test_dataset = []
test_label = []
test_length = []

for i in range(5):
    train_dataset.append([dataset[i*unit : (i+3)*unit]])
    train_label.append([label[i*unit : (i+3)*unit]])
    train_length.append([length[i*unit : (i+3)*unit]])

    dev_dataset.append([dataset[(i+3)*unit : (i+4)*unit]])
    dev_label.append([label[(i+3)*unit : (i+4)*unit]])
    dev_length.append([length[(i+3)*unit : (i+4)*unit]])

    test_dataset.append([dataset[(i+4)*unit : (i+5)*unit]])
    test_label.append([label[(i+4)*unit : (i+5)*unit]])
    test_length.append([length[(i+4)*unit : (i+5)*unit]])

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/ASAPAES')
import score

Generate training data with batch size

In [ ]:
def batch_iter(gold_data, gold_labels, gold_lengths, batch_size, num_epochs):

    assert len(gold_data) == len(gold_labels) == len(gold_lengths)
    data_size = len(gold_data)

    epoch_length = data_size // batch_size
  
    for _ in range(num_epochs):
        for i in range(epoch_length):
            start_index = i * batch_size
            end_index = start_index + batch_size

            xdata = gold_data[start_index: end_index]
            ydata = gold_labels[start_index: end_index]
            sequence_length = gold_lengths[start_index: end_index]

            
            yield xdata, ydata, sequence_length

Bi-LSTM Model

In [ ]:
class Bi_LSTM_Model(object):
    def __init__(self):
        # Score range： 2～12 
        self.hidden_size = 1024
        self.num_layers = 1
        self.l2_reg_lambda = 0.001
        
        # Placeholders
        self.batch_size = tf.placeholder(dtype=tf.int32, shape=[], name='batch_size')
        self.input_x = tf.placeholder(dtype=tf.float32, shape=[None, 75, 768], name='input_x')
        self.input_y = tf.placeholder(dtype=tf.int64, shape=[None], name='input_y')
        self.keep_prob = tf.placeholder(dtype=tf.float32, shape=[], name='keep_prob')
        self.sequence_length = tf.placeholder(dtype=tf.int32, shape=[None], name='sequence_length')

        # L2 loss
        self.l2_loss = tf.constant(0.0)

        # Word embedding
        with tf.name_scope('embedding'):
            inputs = self.input_x

        # Input dropout
        self.inputs = tf.nn.dropout(inputs, keep_prob=self.keep_prob)

        self.final_state = self.bi_lstm()
        
        # Softmax output layer
        with tf.name_scope('sigmoid'):
     
            sigmoid_w = tf.get_variable('sigmoid_w', shape=[2 * self.hidden_size, 13], dtype=tf.float32)
            sigmoid_b = tf.get_variable('sigmoid_b', shape=[13])

            # L2 regularization for output layer
            self.l2_loss += tf.nn.l2_loss(sigmoid_w)
            self.l2_loss += tf.nn.l2_loss(sigmoid_b)
            
            self.logits = tf.matmul(self.final_state, sigmoid_w) + sigmoid_b
            predictions = tf.nn.softmax(self.logits)
            self.predictions = tf.argmax(predictions, 1, name='predictions')

        # Loss
        with tf.name_scope('loss'):
            tvars = tf.trainable_variables()

            # L2 regularization for LSTM weights
            for tv in tvars:
                if 'kernel' in tv.name:
                    self.l2_loss += tf.nn.l2_loss(tv)
            losses = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=self.input_y,
                                                                    logits=self.logits)
            self.cost = tf.reduce_mean(losses, name="loss") + self.l2_reg_lambda * self.l2_loss

        # Accuracy
        with tf.name_scope('accuracy'):
            correct_predictions = tf.equal(self.predictions, self.input_y)
            self.correct_num = tf.reduce_sum(tf.cast(correct_predictions, tf.float32))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32), name='accuracy')


    def bi_lstm(self):
        cell_fw = tf.contrib.rnn.LSTMCell(self.hidden_size,
                                          forget_bias=1.0,
                                          state_is_tuple=True,
                                          reuse=tf.get_variable_scope().reuse)
        cell_bw = tf.contrib.rnn.LSTMCell(self.hidden_size,
                                          forget_bias=1.0,
                                          state_is_tuple=True,
                                          reuse=tf.get_variable_scope().reuse)

        # Add dropout to cell output
        cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, output_keep_prob=self.keep_prob)
        cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, output_keep_prob=self.keep_prob)

        # Stacked LSTMs
        cell_fw = tf.contrib.rnn.MultiRNNCell([cell_fw], state_is_tuple=True)
        cell_bw = tf.contrib.rnn.MultiRNNCell([cell_bw], state_is_tuple=True)

        self._initial_state_fw = cell_fw.zero_state(self.batch_size, dtype=tf.float32)
        self._initial_state_bw = cell_bw.zero_state(self.batch_size, dtype=tf.float32)

        # Dynamic Bi-LSTM
        with tf.variable_scope('Bi-LSTM'):
            _, state = tf.nn.bidirectional_dynamic_rnn(cell_fw,
                                                       cell_bw,
                                                       inputs=self.inputs,
                                                       initial_state_fw=self._initial_state_fw,
                                                       initial_state_bw=self._initial_state_bw,
                                                       sequence_length=self.sequence_length)

        state_fw = state[0]
        state_bw = state[1]
        output = tf.concat([state_fw[self.num_layers - 1].h, state_bw[self.num_layers - 1].h], 1)

        return output

Train Model

In [ ]:
# Model hyperparameters (LSTMs are all single layer)
hidden_size = 1024 # Number of hidden units in the LSTM cell
keep_prob = 0.5 # Dropout keep probability
learning_rate = 1e-5 
l2_reg_lambda = 0.001 # L2 regularization lambda

# Training parameters
batch_size = 256
num_epochs = 100
decay_rate = 1 
decay_steps = 100000 # Learning rate decay rate. Range: (0, 1]
save_every_steps = 100
evaluate_every_steps = 10 # Evaluate the model on validation set after this many steps
num_checkpoint = 50 # number of models to store

for i in range(4, 5):
    outdir = os.path.abspath(os.path.join(os.path.curdir, "Prompt-related Model", "runs", "Bi-LSTM", "Model_" + str(i)))
    if not os.path.exists(outdir):
        os.makedirs(outdir)

    # Load and save data
    # =============================================================================

    # Simple Cross validation
    # Batch iterator
    x_gold_train = train_dataset[i][0]
    y_gold_train = train_label[i][0]*12
    x_gold_dev = dev_dataset[i][0]
    y_gold_dev = dev_label[i][0]*12
    train_gold_lengths = train_length[i][0]
    dev_gold_lengths = dev_length[i][0]
 
    train_data = batch_iter(x_gold_train, y_gold_train, train_gold_lengths, batch_size, num_epochs)
  
    # Train
    # =============================================================================

    with tf.Graph().as_default():
        with tf.Session() as sess:
            classifier = Bi_LSTM_Model()
            # Train procedure
            global_step = tf.Variable(0, name='global_step', trainable=False)
            # Learning rate decay
            starter_learning_rate = learning_rate
            learning_rate = tf.train.exponential_decay(starter_learning_rate,
                                                     global_step,
                                                     decay_steps,
                                                     decay_rate,
                                                     staircase=True)
            optimizer = tf.train.AdamOptimizer(learning_rate)
            grads_and_vars = optimizer.compute_gradients(classifier.cost)
            train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

            # Summaries
            loss_summary = tf.summary.scalar('Loss', classifier.cost)
            accuracy_summary = tf.summary.scalar('Accuracy', classifier.accuracy)

            # Train summary
            train_summary_op = tf.summary.merge_all()
            train_summary_dir = os.path.join(outdir, 'summaries', 'train')
            train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

            # Validation summary
            valid_summary_op = tf.summary.merge_all()
            valid_summary_dir = os.path.join(outdir, 'summaries', 'valid')
            valid_summary_writer = tf.summary.FileWriter(valid_summary_dir, sess.graph)

            saver = tf.train.Saver(max_to_keep=num_checkpoint)

            sess.run(tf.global_variables_initializer())

            def run_step(input_data, is_training=True):
            """Run one step of the training process."""
                input_x, input_y, sequence_length = input_data

                fetches = {'step': global_step,
                         'cost': classifier.cost,
                         'accuracy': classifier.accuracy,
                         'learning_rate': learning_rate}
                feed_dict = {classifier.input_x: input_x,
                           classifier.input_y: input_y}
                fetches['final_state'] = classifier.final_state
                fetches['predictions'] = classifier.predictions
                feed_dict[classifier.batch_size] = len(input_x)
                feed_dict[classifier.sequence_length] = sequence_length

                if is_training:
                    fetches['train_op'] = train_op
                    fetches['summaries'] = train_summary_op
                    feed_dict[classifier.keep_prob] = keep_prob
                else:
                    fetches['summaries'] = valid_summary_op
                    feed_dict[classifier.keep_prob] = 1.0

                vars = sess.run(fetches, feed_dict)
                step = vars['step']
                cost = vars['cost']
                accuracy = vars['accuracy']
                summaries = vars['summaries']
                predictions = vars['predictions']

                # Write summaries to file
                if is_training:
                    train_summary_writer.add_summary(summaries, step)
                else:
                    qwks = score.quadratic_weighted_kappa(predictions, dev_input[1].astype(int), 0, 12)
                    print("QWK ", qwks, " ", score.mean_quadratic_weighted_kappa([qwks]))
                    valid_summary_writer.add_summary(summaries, step)

                time_str = datetime.datetime.now().isoformat()
                print("{}: step: {}, loss: {:g}, accuracy: {:g}".format(time_str, step, cost, accuracy))

                return accuracy


            print('Start training ...')

            for train_input in train_data:
                run_step(train_input, is_training=True)
                current_step = tf.train.global_step(sess, global_step)

                if current_step % evaluate_every_steps == 0:
                    print('\nDevlopment Set Validation')
                    dev_data = batch_iter(x_gold_dev, y_gold_dev, dev_gold_lengths,
                                        batch_size, 1)
                    for dev_input in dev_data:
                        run_step(dev_input, is_training=False)
                    print('End Development Set Validation\n')

                if current_step % save_every_steps == 0:
                    save_path = saver.save(sess, os.path.join(outdir, 'model/clf'), current_step)

            print('\nAll the files have been saved to {}\n'.format(outdir))


In [ ]:
inference_graph = tf.Graph()
with tf.Session(graph = inference_graph) as sess:
    graph = tf.get_default_graph()
    path = '/content/drive/My Drive/Prompt-related Model/runs/Bi-LSTM/'

    for i in range(3,4):
        print("No." + str(i) + " Model\n")
        bilstm_model = tf.train.import_meta_graph(os.path.join(path, "Model_"+str(i), "model", "clf-500.meta"))
        bilstm_model.restore(sess, tf.train.latest_checkpoint(os.path.join(path, "Model_"+str(i), "model")))
    
        x_dev = test_dataset[i][0]
        y_dev = test_label[i][0]*12
        dev_lengths = test_length[i][0]

        print('\nDevlopment Set Validation ' + str(i))
        dev_data = batch_iter(x_dev, y_dev, dev_lengths, 256, 1)
        for dev_input in dev_data:
      
            x_ = inference_graph.get_tensor_by_name('input_x:0')
            y_ = inference_graph.get_tensor_by_name('input_y:0')
            prediction_ = inference_graph.get_tensor_by_name('sigmoid/predictions:0')
            keep_prob_ = inference_graph.get_tensor_by_name('keep_prob:0')
            loss_ = inference_graph.get_tensor_by_name('loss/add_2:0')
            accuracy_ = inference_graph.get_tensor_by_name('accuracy/accuracy:0')
            sequence_length_ = inference_graph.get_tensor_by_name('sequence_length:0')
            batch_size_ = inference_graph.get_tensor_by_name('batch_size:0')
            vars = sess.run([accuracy_, loss_, prediction_], 
                      feed_dict={x_: dev_input[0],
                                 y_: dev_input[1],
                                 keep_prob_: 1.0,
                                 sequence_length_: dev_input[2],
                                 batch_size_ : 256})
            time_str = datetime.datetime.now().isoformat()
            print(vars[2], y_dev)
            qwks = score.quadratic_weighted_kappa(vars[2], (dev_input[1]).astype(int), 0, 12)
            print("qwks",qwks," ", score.mean_quadratic_weighted_kappa([qwks]))
            acc = (np.sum(((vars[2]-dev_input[1])==0)==True)/len(vars[2]))
            print("{}: loss: {:g}, accuracy: {:g}".format(time_str, vars[1], acc))
            print('End Development Set Validation ' + str(i) +'\n')

LSTM Model

In [ ]:
class LSTM_Model(object):
    def __init__(self): 
        self.num_classes = 13
        self.hidden_size = 1024
        self.num_layers = 1
        self.l2_reg_lambda = 0.001
        
        # Placeholders
        self.batch_size = tf.placeholder(dtype=tf.int32, shape=[], name='batch_size')
        self.input_x = tf.placeholder(dtype=tf.float32, shape=[None, 75, 768], name='input_x')
        self.input_y = tf.placeholder(dtype=tf.int64, shape=[None], name='input_y')
        self.keep_prob = tf.placeholder(dtype=tf.float32, shape=[], name='keep_prob')
        self.sequence_length = tf.placeholder(dtype=tf.int32, shape=[None], name='sequence_length')

        # L2 loss
        self.l2_loss = tf.constant(0.0)

        # Word embedding
        with tf.name_scope('embedding'):
            inputs = self.input_x


        # Input dropout
        self.inputs = tf.nn.dropout(inputs, keep_prob=self.keep_prob)

        self.final_state = self.lstm()
        
        # Softmax output layer
        with tf.name_scope('sigmoid'):
            # softmax_w = tf.get_variable('softmax_w', shape=[self.hidden_size, self.num_classes], dtype=tf.float32)
            sigmoid_w = tf.get_variable('sigmoid_w', shape=[self.hidden_size, self.num_classes], dtype=tf.float32)
            sigmoid_b = tf.get_variable('sigmoid_b', shape=[self.num_classes])

            # L2 regularization for output layer
            self.l2_loss += tf.nn.l2_loss(sigmoid_w)
            self.l2_loss += tf.nn.l2_loss(sigmoid_b)
            
            self.logits = tf.matmul(self.final_state, sigmoid_w) + sigmoid_b
            predictions = tf.nn.softmax(self.logits)
            self.predictions = tf.argmax(predictions, 1, name='predictions')

        # Loss (MSE)
        with tf.name_scope('loss'):
            tvars = tf.trainable_variables()

            # L2 regularization for LSTM weights
            for tv in tvars:
                if 'kernel' in tv.name:
                    self.l2_loss += tf.nn.l2_loss(tv)
            losses = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=self.input_y,
                                                                    logits=self.logits)
            self.cost = tf.reduce_mean(losses, name="loss") + self.l2_reg_lambda * self.l2_loss


        # Accuracy
        with tf.name_scope('accuracy'):
            correct_predictions = tf.equal(self.predictions, self.input_y)
            self.correct_num = tf.reduce_sum(tf.cast(correct_predictions, tf.float32))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32), name='accuracy')


    def lstm(self):

        cell = tf.contrib.rnn.LSTMCell(self.hidden_size,
                                          forget_bias= 1.0,
                                          state_is_tuple=True,
                                          reuse=tf.get_variable_scope().reuse)


        # Add dropout to cell output
        cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=self.keep_prob)

        # Stacked LSTMs
        cell = tf.contrib.rnn.MultiRNNCell([cell], state_is_tuple=True)

        self._initial_state = cell.zero_state(self.batch_size, dtype=tf.float32)

        # Dynamic LSTM
        with tf.variable_scope('LSTM'):
            _, state = tf.nn.dynamic_rnn(cell,
                                         inputs=self.inputs,
                                         initial_state=self._initial_state,
                                         sequence_length=self.sequence_length)

        output = state[self.num_layers - 1].h

        return output

In [ ]:
# Model hyperparameters (LSTMs are all single layer)
hidden_size = 1024 # Number of hidden units in the LSTM cell
keep_prob = 0.5 # Dropout keep probability
learning_rate = 1e-5 
l2_reg_lambda = 0.001 # L2 regularization lambda

# Training parameters
batch_size = 256
num_epochs = 100
decay_rate = 1 
decay_steps = 100000 # Learning rate decay rate. Range: (0, 1]
save_every_steps = 100
evaluate_every_steps = 10 # Evaluate the model on validation set after this many steps
num_checkpoint = 50 # number of models to store

for i in range(4, 5):
    outdir = os.path.abspath(os.path.join(os.path.curdir, "Prompt-related Model", "runs", "LSTM", "Model_"+str(i)))
    if not os.path.exists(outdir):
        os.makedirs(outdir)

    # Load and save data
    # =============================================================================

    # Simple Cross validation
    # Batch iterator
    x_gold_train = train_dataset[i][0]
    y_gold_train = train_label[i][0]*12
    x_gold_dev = dev_dataset[i][0]
    y_gold_dev = dev_label[i][0]*12
    train_gold_lengths = train_length[i][0]
    dev_gold_lengths = dev_length[i][0]
 
    train_data = batch_iter(x_gold_train, y_gold_train, train_gold_lengths, batch_size, num_epochs)
  
  
    # Train
    # =============================================================================

    with tf.Graph().as_default():
        with tf.Session() as sess:
            classifier = LSTM_Model()
            # Train procedure
            global_step = tf.Variable(0, name='global_step', trainable=False)
            # Learning rate decay
            starter_learning_rate = learning_rate
            learning_rate = tf.train.exponential_decay(starter_learning_rate,
                                                     global_step,
                                                     decay_steps,
                                                     decay_rate,
                                                     staircase=True)
            optimizer = tf.train.AdamOptimizer(learning_rate)
            grads_and_vars = optimizer.compute_gradients(classifier.cost)
            train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

            # Summaries
            loss_summary = tf.summary.scalar('Loss', classifier.cost)
            accuracy_summary = tf.summary.scalar('Accuracy', classifier.accuracy)

            # Train summary
            train_summary_op = tf.summary.merge_all()
            train_summary_dir = os.path.join(outdir, 'summaries', 'train')
            train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

            # Validation summary
            valid_summary_op = tf.summary.merge_all()
            valid_summary_dir = os.path.join(outdir, 'summaries', 'valid')
            valid_summary_writer = tf.summary.FileWriter(valid_summary_dir, sess.graph)

            saver = tf.train.Saver(max_to_keep=num_checkpoint)

            sess.run(tf.global_variables_initializer())


            def run_step(input_data, is_training=True):
            """Run one step of the training process."""
                input_x, input_y, sequence_length = input_data

                fetches = {'step': global_step,
                         'cost': classifier.cost,
                         'accuracy': classifier.accuracy,
                         'learning_rate': learning_rate}
                feed_dict = {classifier.input_x: input_x,
                           classifier.input_y: input_y}
                fetches['final_state'] = classifier.final_state
                fetches['predictions'] = classifier.predictions
                feed_dict[classifier.batch_size] = len(input_x)
                feed_dict[classifier.sequence_length] = sequence_length

                if is_training:
                fetches['train_op'] = train_op
                  fetches['summaries'] = train_summary_op
                  feed_dict[classifier.keep_prob] = keep_prob
              else:
                  fetches['summaries'] = valid_summary_op
                  feed_dict[classifier.keep_prob] = 1.0

              vars = sess.run(fetches, feed_dict)
              step = vars['step']
              cost = vars['cost']
              predictions = vars['predictions']
              accuracy = vars['accuracy']
              summaries = vars['summaries']

              # Write summaries to file
              if is_training:
                  train_summary_writer.add_summary(summaries, step)
              else:
                  qwks = score.quadratic_weighted_kappa(predictions, (dev_input[1]).astype(int), 0, 12)
                  print("qwks",qwks," ", score.mean_quadratic_weighted_kappa([qwks]))
                  valid_summary_writer.add_summary(summaries, step)

              time_str = datetime.datetime.now().isoformat()
              print("{}: step: {}, loss: {:g}, accuracy: {:g}".format(time_str, step, cost, accuracy))

              return accuracy


          print('Start training ...')

          for train_input in train_data:
              run_step(train_input, is_training=True)
              current_step = tf.train.global_step(sess, global_step)

              if current_step % evaluate_every_steps == 0:
                  print('\nDevlopment Set Validation')
                  dev_data = batch_iter(x_gold_dev, y_gold_dev, dev_gold_lengths,
                                        batch_size, 1)
                  for dev_input in dev_data:
                    run_step(dev_input, is_training=False)
                  print('End Development Set Validation\n')

              if current_step % save_every_steps == 0:
                  save_path = saver.save(sess, os.path.join(outdir, 'model/clf'), current_step)

          print('\nAll the files have been saved to {}\n'.format(outdir))


In [ ]:
inference_graph = tf.Graph()
with tf.Session(graph = inference_graph) as sess:
  
  graph = tf.get_default_graph()
  path = '/content/drive/My Drive/Prompt-related Model/runs/LSTM/'

  
  for i in range(3,4):
    print("No." + str(i) + " Model\n")
    bilstm_model = tf.train.import_meta_graph(os.path.join(path, "Model_"+str(i), "model", "clf-500.meta"))
    bilstm_model.restore(sess, tf.train.latest_checkpoint(os.path.join(path, "Model_"+str(i), "model")))
#     print ([n.name for n in graph.as_graph_def().node])
    
    x_dev = test_dataset[i][0]
    y_dev = test_label[i][0]*12
    dev_lengths = test_length[i][0]

    print('\nDevlopment Set Validation ' + str(i))
    dev_data = batch_iter(x_dev, y_dev, dev_lengths, 256, 1)
    for dev_input in dev_data:
      
      x_ = inference_graph.get_tensor_by_name('input_x:0')
      y_ = inference_graph.get_tensor_by_name('input_y:0')
      prediction_ = inference_graph.get_tensor_by_name('sigmoid/predictions:0')
      keep_prob_ = inference_graph.get_tensor_by_name('keep_prob:0')
      loss_ = inference_graph.get_tensor_by_name('loss/loss:0')
      accuracy_ = inference_graph.get_tensor_by_name('accuracy/accuracy:0')
      sequence_length_ = inference_graph.get_tensor_by_name('sequence_length:0')
      batch_size_ = inference_graph.get_tensor_by_name('batch_size:0')
      vars = sess.run([accuracy_, loss_, prediction_], 
                      feed_dict={x_: dev_input[0],
                                 y_: dev_input[1],
                                 keep_prob_: 1.0,
                                 sequence_length_: dev_input[2],
                                 batch_size_ : 256})
      time_str = datetime.datetime.now().isoformat()
      print(vars[2], y_dev)
      qwks = score.quadratic_weighted_kappa(vars[2], (dev_input[1]).astype(int), 0, 12)
      print("qwks",qwks," ", score.mean_quadratic_weighted_kappa([qwks]))
      acc = (np.sum(((vars[2]-dev_input[1])==0)==True)/len(vars[2]))
      print("{}: loss: {:g}, accuracy: {:g}".format(time_str, vars[1], acc))
      print('End Development Set Validation ' + str(i) +'\n')